In [5]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score
from sklearn.feature_selection import RFE
import catboost as cb
from tqdm import tqdm

ROOT_DIR = "data"
RANDOM_STATE = 110

train_data = pd.read_csv(os.path.join(ROOT_DIR, "train_mod.csv"))

def cat2num(X):
    non_numeric_columns = X.select_dtypes(include=['object']).columns
    # print("Non-numeric columns:", non_numeric_columns)

    encoded_columns = {}

    for column in non_numeric_columns:
        encoder = LabelEncoder()
        encoded_columns[column] = encoder.fit_transform(X[column])

    encoded_df = pd.DataFrame(encoded_columns, index=X.index)

    X = X.drop(columns=non_numeric_columns)
    X = pd.concat([X, encoded_df], axis=1)

    return X

train_data = cat2num(train_data)
train_data = train_data[['Receip No Collect Result_Dam', '1st Pressure 1st Pressure Unit Time_AutoClave', '3rd Pressure Collect Result_AutoClave', 'Chamber Temp. Unit Time_AutoClave', 'Receip No Collect Result_Fill1', 'Receip No Collect Result_Fill2',
                         'Head Clean Position Z Collect Result_Dam',
                         'target']]

# 'Head Clean Position Z Collect Result_Dam' 컬럼을 120 이상인 값과 이하인 값으로 이진분류
train_data['Head Clean Position Z Collect Result_Dam'] = train_data['Head Clean Position Z Collect Result_Dam'] >= 120

# 이진분류 결과가 True/False로 나올 텐데, 이를 1과 0으로 바꾸려면 다음과 같이 할 수 있습니다.
train_data['Head Clean Position Z Collect Result_Dam'] = train_data['Head Clean Position Z Collect Result_Dam'].astype(int)

# Scale the data
scaler = MinMaxScaler()
columns_to_scale = [col for col in train_data.columns if col != 'target']
train_data[columns_to_scale] = scaler.fit_transform(train_data[columns_to_scale])

# Undersample the data
normal_ratio = 1.0  # 1:1 ratio
df_normal = train_data[train_data["target"] == 1]
df_abnormal = train_data[train_data["target"] == 0]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

# Split the data into features and target
train_x = df_concat.drop(columns=['target'])
train_y = df_concat['target']

X_train, X_test, y_train, y_test = train_test_split(
    train_x,
    train_y,
    test_size=0.3,
    random_state=RANDOM_STATE,
)

# Recursive Feature Elimination (RFE) for backward feature selection
model = cb.CatBoostClassifier(
    depth=6,
    iterations=400,
    l2_leaf_reg=5,
    learning_rate=0.01,
    verbose=0  # No output during training
)

# Train final model
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")

# Process test data
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test_mod.csv"))
test_data = cat2num(test_data)

# Scale the test data
test_data[columns_to_scale] = scaler.transform(test_data[columns_to_scale])

# Select the same features as the training data
test_x_rfe = test_data[X_train.columns]

# Predict on test data
y_pred = model.predict(test_x_rfe)

# Prepare submission
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = y_pred
df_sub['target'] = df_sub['target'].map({1: 'Normal', 0: 'AbNormal'})

# Calculate the ratio of abnormal cases
counts = df_sub['target'].value_counts()
ratio = counts['AbNormal'] / (counts['AbNormal'] + counts['Normal'])
print("The ratio of abnormal is:", ratio)

# Save the submission file
df_sub.to_csv("submission.csv", index=False)

Total: Normal: 38156, AbNormal: 2350
F1 Score: 0.6694
Recall: 0.7940
Accuracy: 0.5950
Precision: 0.5786
The ratio of abnormal is: 0.23362709521340935


In [4]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score
from sklearn.feature_selection import SequentialFeatureSelector
import catboost as cb
from tqdm import tqdm

ROOT_DIR = "data"
RANDOM_STATE = 110

train_data = pd.read_csv(os.path.join(ROOT_DIR, "train_mod.csv"))

def cat2num(X):
    non_numeric_columns = X.select_dtypes(include=['object']).columns
    # print("Non-numeric columns:", non_numeric_columns)

    encoded_columns = {}

    for column in non_numeric_columns:
        encoder = LabelEncoder()
        encoded_columns[column] = encoder.fit_transform(X[column])

    encoded_df = pd.DataFrame(encoded_columns, index=X.index)

    X = X.drop(columns=non_numeric_columns)
    X = pd.concat([X, encoded_df], axis=1)

    return X

train_data = cat2num(train_data)
train_data = train_data[['Receip No Collect Result_Dam', '1st Pressure 1st Pressure Unit Time_AutoClave', '3rd Pressure Collect Result_AutoClave', 'Chamber Temp. Unit Time_AutoClave', 'Receip No Collect Result_Fill1', 'Receip No Collect Result_Fill2', 'target']]

# Scale the data
scaler = MinMaxScaler()
columns_to_scale = [col for col in train_data.columns if col != 'target']
train_data[columns_to_scale] = scaler.fit_transform(train_data[columns_to_scale])

# Undersample the data
normal_ratio = 1.0  # 1:1 ratio
df_normal = train_data[train_data["target"] == 1]
df_abnormal = train_data[train_data["target"] == 0]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

# Split the data into features and target
train_x = df_concat.drop(columns=['target'])
train_y = df_concat['target']

X_train, X_test, y_train, y_test = train_test_split(
    train_x,
    train_y,
    test_size=0.3,
    random_state=RANDOM_STATE,
)

# CatBoost classifier for SFS
model = cb.CatBoostClassifier(
    depth=6,
    iterations=400,
    l2_leaf_reg=5,
    learning_rate=0.01,
    verbose=0  # No output during training
)

# Sequential Feature Selector (SFS)
sfs = SequentialFeatureSelector(
    model,
    n_features_to_select="auto",  # Auto will select a feature set with the best score
    direction='forward',  # Can be 'forward' or 'backward'
    scoring='f1',  # You can use 'f1', 'precision', 'recall', etc.
    cv=5,  # 5-fold cross-validation
    n_jobs=-1  # Use all available CPU cores
)

sfs.fit(X_train, y_train)

# Get selected feature names and apply to both training and test data
selected_features = X_train.columns[sfs.get_support()]
X_train_sfs = X_train[selected_features]
X_test_sfs = X_test[selected_features]

# Train final model on selected features
model.fit(X_train_sfs, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_sfs)

f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")

# Process test data
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test_mod.csv"))
test_data = cat2num(test_data)

# Scale the test data
test_data[columns_to_scale] = scaler.transform(test_data[columns_to_scale])

# Select the same features as the training data
test_x_sfs = test_data[selected_features]

# Predict on test data
y_pred = model.predict(test_x_sfs)

# Prepare submission
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = y_pred
df_sub['target'] = df_sub['target'].map({1: 'Normal', 0: 'AbNormal'})

# Calculate the ratio of abnormal cases
counts = df_sub['target'].value_counts()
ratio = counts['AbNormal'] / (counts['AbNormal'] + counts['Normal'])
print("The ratio of abnormal is:", ratio)

# Save the submission file
df_sub.to_csv("submission.csv", index=False)

Non-numeric columns: Index(['Wip Line_Dam', 'Process Desc._Dam', 'Equipment_Dam',
       'Model.Suffix_Dam', 'Workorder_Dam', 'Insp Judge Code_Dam',
       'Wip Line_AutoClave', 'Process Desc._AutoClave', 'Equipment_AutoClave',
       'Model.Suffix_AutoClave', 'Workorder_AutoClave',
       'Insp Judge Code_AutoClave', '1st Pressure Judge Value_AutoClave',
       '2nd Pressure Judge Value_AutoClave',
       '3rd Pressure Judge Value_AutoClave',
       'Chamber Temp. Judge Value_AutoClave', 'Wip Line_Fill1',
       'Process Desc._Fill1', 'Equipment_Fill1', 'Model.Suffix_Fill1',
       'Workorder_Fill1', 'Insp Judge Code_Fill1', 'Wip Line_Fill2',
       'Process Desc._Fill2', 'Equipment_Fill2', 'Model.Suffix_Fill2',
       'Workorder_Fill2', 'Insp Judge Code_Fill2', 'target'],
      dtype='object')
Total: Normal: 38156, AbNormal: 2350
F1 Score: 0.6686
Recall: 0.8008
Accuracy: 0.5901
Precision: 0.5738
Non-numeric columns: Index(['Set ID', 'Wip Line_Dam', 'Process Desc._Dam', 'Equipment_Dam

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score
from sklearn.feature_selection import RFE
import catboost as cb
from tqdm import tqdm

ROOT_DIR = "data"
RANDOM_STATE = 110

train_data = pd.read_csv(os.path.join(ROOT_DIR, "train_mod.csv"))

def cat2num(X):
    non_numeric_columns = X.select_dtypes(include=['object']).columns
    # print("Non-numeric columns:", non_numeric_columns)

    encoded_columns = {}

    for column in non_numeric_columns:
        encoder = LabelEncoder()
        encoded_columns[column] = encoder.fit_transform(X[column])

    encoded_df = pd.DataFrame(encoded_columns, index=X.index)

    X = X.drop(columns=non_numeric_columns)
    X = pd.concat([X, encoded_df], axis=1)

    return X

train_data = cat2num(train_data)
train_data = train_data[['Receip No Collect Result_Dam', '1st Pressure 1st Pressure Unit Time_AutoClave', '3rd Pressure Collect Result_AutoClave', 'Chamber Temp. Unit Time_AutoClave', 'Receip No Collect Result_Fill1', 'Receip No Collect Result_Fill2', 'target']]

# Scale the data
scaler = MinMaxScaler()
columns_to_scale = [col for col in train_data.columns if col != 'target']
train_data[columns_to_scale] = scaler.fit_transform(train_data[columns_to_scale])

# Undersample the data
normal_ratio = 1.0  # 1:1 ratio
df_normal = train_data[train_data["target"] == 1]
df_abnormal = train_data[train_data["target"] == 0]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

# Split the data into features and target
train_x = df_concat.drop(columns=['target'])
train_y = df_concat['target']

X_train, X_test, y_train, y_test = train_test_split(
    train_x,
    train_y,
    test_size=0.3,
    random_state=RANDOM_STATE,
)

# CatBoost classifier for RFE
model = cb.CatBoostClassifier(
    depth=6,
    iterations=400,
    l2_leaf_reg=5,
    learning_rate=0.01,
    verbose=0  # No output during training
)

# Recursive Feature Elimination (RFE)
rfe = RFE(
    estimator=model,
    n_features_to_select=5,  # Adjust based on how many features you want to select
    step=1  # Number of features to remove at each iteration
)

# Fit RFE
rfe.fit(X_train, y_train)

# Get selected features and apply to both training and test data
selected_features = X_train.columns[rfe.support_]
X_train_rfe = X_train[selected_features]
X_test_rfe = X_test[selected_features]

# Train final model on selected features
model.fit(X_train_rfe, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_rfe)

f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")

# Process test data
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test_mod.csv"))
test_data = cat2num(test_data)

# Scale the test data
test_data[columns_to_scale] = scaler.transform(test_data[columns_to_scale])

# Select the same features as the training data
test_x_rfe = test_data[selected_features]

# Predict on test data
y_pred = model.predict(test_x_rfe)

# Prepare submission
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = y_pred
df_sub['target'] = df_sub['target'].map({1: 'Normal', 0: 'AbNormal'})

# Calculate the ratio of abnormal cases
counts = df_sub['target'].value_counts()
ratio = counts['AbNormal'] / (counts['AbNormal'] + counts['Normal'])
print("The ratio of abnormal is:", ratio)

# Save the submission file
df_sub.to_csv("submission.csv", index=False)


In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector
import catboost as cb
from tqdm import tqdm

ROOT_DIR = "data"
RANDOM_STATE = 110

train_data = pd.read_csv(os.path.join(ROOT_DIR, "train_mod.csv"))

def cat2num(X):
    non_numeric_columns = X.select_dtypes(include=['object']).columns
    # print("Non-numeric columns:", non_numeric_columns)

    encoded_columns = {}

    for column in non_numeric_columns:
        encoder = LabelEncoder()
        encoded_columns[column] = encoder.fit_transform(X[column])

    encoded_df = pd.DataFrame(encoded_columns, index=X.index)

    X = X.drop(columns=non_numeric_columns)
    X = pd.concat([X, encoded_df], axis=1)

    return X

def generating_features(df):
    # Feature engineering for thickness and pressure
    df['Thickness_Diff_1_2'] = df['THICKNESS 1 Collect Result_Dam'] - df['THICKNESS 2 Collect Result_Dam']
    df['Thickness_Diff_2_3'] = df['THICKNESS 2 Collect Result_Dam'] - df['THICKNESS 3 Collect Result_Dam']
    df['Thickness_Diff_1_3'] = df['THICKNESS 1 Collect Result_Dam'] - df['THICKNESS 3 Collect Result_Dam']

    df['Thickness_Avg'] = (
        df['THICKNESS 1 Collect Result_Dam'] +
        df['THICKNESS 2 Collect Result_Dam'] +
        df['THICKNESS 3 Collect Result_Dam']) / 3

    df['Thickness_Std'] = df[['THICKNESS 1 Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam', 'THICKNESS 3 Collect Result_Dam']].std(axis=1)

    df['Pressure_Diff_1st_2nd'] = df['1st Pressure Collect Result_AutoClave'] - df['2nd Pressure Collect Result_AutoClave']
    df['Pressure_Diff_2nd_3rd'] = df['2nd Pressure Collect Result_AutoClave'] - df['3rd Pressure Collect Result_AutoClave']
    df['Pressure_Diff_1st_3rd'] = df['1st Pressure Collect Result_AutoClave'] - df['3rd Pressure Collect Result_AutoClave']

    df['Pressure_Avg'] = (
        df['1st Pressure Collect Result_AutoClave'] +
        df['2nd Pressure Collect Result_AutoClave'] +
        df['3rd Pressure Collect Result_AutoClave']) / 3

    df['Pressure_Std'] = df[['1st Pressure Collect Result_AutoClave', '2nd Pressure Collect Result_AutoClave', '3rd Pressure Collect Result_AutoClave']].std(axis=1)

    return df

train_data = cat2num(train_data)
train_data = generating_features(train_data)
train_data = train_data[['Receip No Collect Result_Dam', '1st Pressure 1st Pressure Unit Time_AutoClave', '3rd Pressure Collect Result_AutoClave', 'Chamber Temp. Unit Time_AutoClave', 'Receip No Collect Result_Fill1', 'Receip No Collect Result_Fill2', 
                         'Thickness_Diff_1_2', 'Thickness_Diff_2_3', 'Thickness_Diff_1_3', 'Thickness_Avg', 'Thickness_Std', 'Pressure_Diff_1st_2nd', 'Pressure_Diff_2nd_3rd', 'Pressure_Diff_1st_3rd', 'Pressure_Avg', 'Pressure_Std',
                         'target']]

# Scale the data
scaler = MinMaxScaler()
columns_to_scale = [col for col in train_data.columns if col != 'target']
train_data[columns_to_scale] = scaler.fit_transform(train_data[columns_to_scale])

# Undersample the data
normal_ratio = 1.0  # 1:1 ratio
df_normal = train_data[train_data["target"] == 1]
df_abnormal = train_data[train_data["target"] == 0]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

# Split the data into features and target
train_x = df_concat.drop(columns=['target'])
train_y = df_concat['target']

X_train, X_test, y_train, y_test = train_test_split(
    train_x,
    train_y,
    test_size=0.3,
    random_state=RANDOM_STATE,
)

# CatBoost classifier for SFS
model = cb.CatBoostClassifier(
    depth=6,
    iterations=400,
    l2_leaf_reg=5,
    learning_rate=0.01,
    verbose=0  # No output during training
)

# Sequential Feature Selector (SFS)
sfs = SequentialFeatureSelector(
    model,
    n_features_to_select="auto",  # Auto will select a feature set with the best score
    direction='forward',  # Can be 'forward' or 'backward'
    scoring='f1',  # You can use 'f1', 'precision', 'recall', etc.
    cv=5,  # 5-fold cross-validation
    n_jobs=-1  # Use all available CPU cores
)

sfs.fit(X_train, y_train)

# Get selected feature names and apply to both training and test data
selected_features = X_train.columns[sfs.get_support()]
X_train_sfs = X_train[selected_features]
X_test_sfs = X_test[selected_features]

# Train final model on selected features
model.fit(X_train_sfs, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_sfs)

f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")

# Process test data
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test_mod.csv"))
test_data = cat2num(test_data)
test_data = generating_features(test_data)

# Scale the test data
test_data[columns_to_scale] = scaler.transform(test_data[columns_to_scale])

# Select the same features as the training data
test_x_sfs = test_data[selected_features]

# Predict on test data
y_pred = model.predict(test_x_sfs)

# Prepare submission
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = y_pred
df_sub['target'] = df_sub['target'].map({1: 'Normal', 0: 'AbNormal'})

# Calculate the ratio of abnormal cases
counts = df_sub['target'].value_counts()
ratio = counts['AbNormal'] / (counts['AbNormal'] + counts['Normal'])
print("The ratio of abnormal is:", ratio)

# Save the submission file
df_sub.to_csv("submission.csv", index=False)

Total: Normal: 38156, AbNormal: 2350
F1 Score: 0.6620
Recall: 0.7775
Accuracy: 0.5901
Precision: 0.5764
The ratio of abnormal is: 0.2429007545648292


In [4]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score
from sklearn.feature_selection import RFE
import catboost as cb
from tqdm import tqdm

ROOT_DIR = "data"
RANDOM_STATE = 110

train_data = pd.read_csv(os.path.join(ROOT_DIR, "train_mod.csv"))

def cat2num(X):
    non_numeric_columns = X.select_dtypes(include=['object']).columns
    print("Non-numeric columns:", non_numeric_columns)

    encoded_columns = {}

    for column in non_numeric_columns:
        encoder = LabelEncoder()
        encoded_columns[column] = encoder.fit_transform(X[column])

    encoded_df = pd.DataFrame(encoded_columns, index=X.index)

    X = X.drop(columns=non_numeric_columns)
    X = pd.concat([X, encoded_df], axis=1)

    return X

def generating_features(df):
    # Feature engineering for thickness and pressure
    df['Thickness_Diff_1_2'] = df['THICKNESS 1 Collect Result_Dam'] - df['THICKNESS 2 Collect Result_Dam']
    df['Thickness_Diff_2_3'] = df['THICKNESS 2 Collect Result_Dam'] - df['THICKNESS 3 Collect Result_Dam']
    df['Thickness_Diff_1_3'] = df['THICKNESS 1 Collect Result_Dam'] - df['THICKNESS 3 Collect Result_Dam']

    df['Thickness_Avg'] = (
        df['THICKNESS 1 Collect Result_Dam'] +
        df['THICKNESS 2 Collect Result_Dam'] +
        df['THICKNESS 3 Collect Result_Dam']) / 3

    df['Thickness_Std'] = df[['THICKNESS 1 Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam', 'THICKNESS 3 Collect Result_Dam']].std(axis=1)

    df['Pressure_Diff_1st_2nd'] = df['1st Pressure Collect Result_AutoClave'] - df['2nd Pressure Collect Result_AutoClave']
    df['Pressure_Diff_2nd_3rd'] = df['2nd Pressure Collect Result_AutoClave'] - df['3rd Pressure Collect Result_AutoClave']
    df['Pressure_Diff_1st_3rd'] = df['1st Pressure Collect Result_AutoClave'] - df['3rd Pressure Collect Result_AutoClave']

    df['Pressure_Avg'] = (
        df['1st Pressure Collect Result_AutoClave'] +
        df['2nd Pressure Collect Result_AutoClave'] +
        df['3rd Pressure Collect Result_AutoClave']) / 3

    df['Pressure_Std'] = df[['1st Pressure Collect Result_AutoClave', '2nd Pressure Collect Result_AutoClave', '3rd Pressure Collect Result_AutoClave']].std(axis=1)

    return df

def cart2sph(df):
    coordinate_columns = [
        ("HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam", "HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam", "HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam"),
        ("HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam", "HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam", "HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam"),
        ("HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam", "HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam", "HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam"),
        ("HEAD Standby Position X Collect Result_Dam", "HEAD Standby Position Y Collect Result_Dam", "HEAD Standby Position Z Collect Result_Dam"),
        ("Head Clean Position X Collect Result_Dam", "Head Clean Position Y Collect Result_Dam", "Head Clean Position Z Collect Result_Dam"),
        ("Head Purge Position X Collect Result_Dam", "Head Purge Position Y Collect Result_Dam", "Head Purge Position Z Collect Result_Dam"),
        ("Head Zero Position X Collect Result_Dam", "Head Zero Position Y Collect Result_Dam", "Head Zero Position Z Collect Result_Dam"),

        ("HEAD Standby Position X Collect Result_Fill1", "HEAD Standby Position Y Collect Result_Fill1", "HEAD Standby Position Z Collect Result_Fill1"),
        ("Head Clean Position X Collect Result_Fill1", "Head Clean Position Y Collect Result_Fill1", "Head Clean Position Z Collect Result_Fill1"),
        ("Head Purge Position X Collect Result_Fill1", "Head Purge Position Y Collect Result_Fill1", "Head Purge Position Z Collect Result_Fill1"),
        ("HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1", "HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1", "HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1"),
        ("HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1", "HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1", "HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1"),
        ("HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1", "HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1", "HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1"),

        ("HEAD Standby Position X Collect Result_Fill2", "HEAD Standby Position Y Collect Result_Fill2", "HEAD Standby Position Z Collect Result_Fill2"),
        ("Head Clean Position X Collect Result_Fill2", "Head Clean Position Y Collect Result_Fill2", "Head Clean Position Z Collect Result_Fill2"),
        ("Head Purge Position X Collect Result_Fill2", "Head Purge Position Y Collect Result_Fill2", "Head Purge Position Z Collect Result_Fill2"),
        ("HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2", "HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2", "HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2"),
        ("HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2", "HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2", "HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2"),
        ("HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2", "HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2", "HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2"),
    ]

    cylindrical_columns = [
        ("CURE START POSITION X Collect Result_Dam", "CURE START POSITION Z Collect Result_Dam", "CURE START POSITION Θ Collect Result_Dam"),
        ("CURE STANDBY POSITION X Collect Result_Dam", "CURE STANDBY POSITION Z Collect Result_Dam", "CURE STANDBY POSITION Θ Collect Result_Dam"),
        ("CURE END POSITION X Collect Result_Dam", "CURE END POSITION Z Collect Result_Dam", "CURE END POSITION Θ Collect Result_Dam"),
        ("CURE START POSITION X Collect Result_Fill2", "CURE START POSITION Z Collect Result_Fill2", "CURE START POSITION Θ Collect Result_Fill2"),
        ("CURE STANDBY POSITION X Collect Result_Fill2", "CURE STANDBY POSITION Z Collect Result_Fill2", "CURE STANDBY POSITION Θ Collect Result_Fill2"),
        ("CURE END POSITION X Collect Result_Fill2", "CURE END POSITION Z Collect Result_Fill2", "CURE END POSITION Θ Collect Result_Fill2"),
    ]

    for x_col, y_col, z_col in coordinate_columns:
        df[x_col] = np.sqrt(df[x_col]**2 + df[y_col]**2 + df[z_col]**2)
        df[y_col] = np.arctan2(df[y_col], df[x_col])
        df[z_col] = np.arccos(df[z_col] / df[x_col])

        r_col = x_col.replace("X", "r").replace("Y", "θ").replace("Z", "φ")
        theta_col = y_col.replace("X", "r").replace("Y", "θ").replace("Z", "φ")
        phi_col = z_col.replace("X", "r").replace("Y", "θ").replace("Z", "φ")

        df.rename(columns={x_col: r_col, y_col: theta_col, z_col: phi_col}, inplace=True)

    for x_col, z_col, theta_col in cylindrical_columns:
        df[x_col] = np.sqrt(df[x_col]**2 + df[z_col]**2)
        df[z_col] = np.arctan2(df[z_col], df[x_col])

        r_col = x_col.replace("X", "r").replace("Z", "z").replace("Θ", "θ")
        df.rename(columns={x_col: r_col, z_col: theta_col}, inplace=True)

    return df

def preprocess(df):
    # Remove columns where every value is unique
    same_rows_columns = [column for column in df.columns if df[column].nunique() == 1]
    row_count = len(df)
    matching_row_columns = [column for column in df.columns if df[column].value_counts().size == row_count]

    # columns_to_remove = ['Wip Line_Dam', 'Process Desc._Dam', 'Equipment_Dam', 'Model.Suffix_Dam', 'Workorder_Dam',
    #                      'Wip Line_AutoClave', 'Process Desc._AutoClave', 'Equipment_AutoClave', 'Model.Suffix_AutoClave', 'Workorder_AutoClave',
    #                      'Wip Line_Fill1', 'Process Desc._Fill1', 'Equipment_Fill1', 'Model.Suffix_Fill1', 'Workorder_Fill1',
    #                      'Wip Line_Fill2', 'Process Desc._Fill2', 'Equipment_Fill2', 'Model.Suffix_Fill2', 'Workorder_Fill2']
    columns_to_remove = []

    all_columns_to_remove = list(set(same_rows_columns + matching_row_columns + columns_to_remove))
    df.drop(columns=[col for col in all_columns_to_remove if col in df.columns], inplace=True)

    print("same_rows_columns", same_rows_columns)
    print("matching_row_columns", matching_row_columns)
    return df

train_data = preprocess(train_data)
train_data = cat2num(train_data)
train_data = generating_features(train_data)
train_data = cart2sph(train_data)

# Scale the data
scaler = MinMaxScaler()
columns_to_scale = [col for col in train_data.columns if col != 'target']
train_data[columns_to_scale] = scaler.fit_transform(train_data[columns_to_scale])

# Undersample the data
normal_ratio = 1.0  # 1:1 ratio
df_normal = train_data[train_data["target"] == 1]
df_abnormal = train_data[train_data["target"] == 0]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

# Split the data into features and target
train_x = df_concat.drop(columns=['target'])
train_y = df_concat['target']

X_train, X_test, y_train, y_test = train_test_split(
    train_x,
    train_y,
    test_size=0.3,
    random_state=RANDOM_STATE,
)

# Recursive Feature Elimination (RFE) for backward feature selection
model = cb.CatBoostClassifier(
    depth=6,
    iterations=400,
    l2_leaf_reg=5,
    learning_rate=0.01,
    verbose=0  # No output during training
)

# Train final model
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")

# Process test data
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test_mod.csv"))
test_data = preprocess(test_data)
test_data = cat2num(test_data)
test_data = generating_features(test_data)
test_data = cart2sph(test_data)

# Scale the test data
test_data[columns_to_scale] = scaler.transform(test_data[columns_to_scale])

# Select the same features as the training data
test_x_rfe = test_data[X_train.columns]

# Predict on test data
y_pred = model.predict(test_x_rfe)

# Prepare submission
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = y_pred
df_sub['target'] = df_sub['target'].map({1: 'Normal', 0: 'AbNormal'})

# Calculate the ratio of abnormal cases
counts = df_sub['target'].value_counts()
ratio = counts['AbNormal'] / (counts['AbNormal'] + counts['Normal'])
print("The ratio of abnormal is:", ratio)

# Save the submission file
df_sub.to_csv("submission.csv", index=False)

same_rows_columns ['Wip Line_Dam', 'Process Desc._Dam', 'Insp. Seq No._Dam', 'Insp Judge Code_Dam', 'CURE STANDBY POSITION X Collect Result_Dam', 'CURE STANDBY POSITION Z Collect Result_Dam', 'CURE STANDBY POSITION Θ Collect Result_Dam', 'CURE START POSITION Z Collect Result_Dam', 'HEAD Standby Position X Collect Result_Dam', 'HEAD Standby Position Y Collect Result_Dam', 'HEAD Standby Position Z Collect Result_Dam', 'Head Clean Position X Collect Result_Dam', 'Head Clean Position Y Collect Result_Dam', 'Head Purge Position X Collect Result_Dam', 'Head Purge Position Y Collect Result_Dam', 'Head Zero Position X Collect Result_Dam', 'WorkMode Collect Result_Dam', 'Wip Line_AutoClave', 'Process Desc._AutoClave', 'Equipment_AutoClave', 'Insp. Seq No._AutoClave', 'Insp Judge Code_AutoClave', '1st Pressure Judge Value_AutoClave', '2nd Pressure Judge Value_AutoClave', '3rd Pressure Judge Value_AutoClave', 'Wip Line_Fill1', 'Process Desc._Fill1', 'Insp. Seq No._Fill1', 'Insp Judge Code_Fill1',

KeyError: 'HEAD Standby Position X Collect Result_Dam'